# 房贷还款计算器

用于对房贷还款计划做决策。对比还款前后总支付金额。同时对比将资金用于其他投资产生的收益情况。

投资有风险，决策须谨慎，投资收益的计算结果是假设无风险且收益率可保证在设置的最低水平，且可以至少持续到贷款结束的时间。

## 基础知识

### 机会成本

机会成本是指在做出某种选择后，次优选项所带来的损失。

### 复利

复利的计算方法是：

$$
F = P \times (1 + r)^n
$$

其中，$F$ 是最终金额，$P$ 是本金，$r$ 是利率，$n$ 是计息期数。如果 $r$ 是年利率，则期数$n$是计息年数。

## 贷款信息设置

In [88]:
import pandas as pd
from IPython.display import Markdown


# 贷款剩余本金
principal_before = 10000
# 年利率
rate_year = 3.9/100
# 贷款剩余期数
term = 36

# 提前还款的本金
return_amount = 5000
# 提前还款支付的手续费、利息
return_intrest = 5

# 机会成本计算(年利率)
invest_intrest_year = 2/100


# 月利率计算
rate_month = rate_year/12
invest_intrest_month = invest_intrest_year/12

## 等额本金

In [89]:
decrease_before = principal_before / term
decrease_after = (principal_before - return_amount) / term
remain_before = principal_before
remain_after = principal_before - return_amount
payments_data = []

total_payments_before = (
    principal_before * rate_month + decrease_before * rate_month
) * term / 2 + principal_before

total_payments_after = (
    (remain_after * rate_month + decrease_after * rate_month) * term / 2
    + remain_after
    + return_amount
    + return_intrest
)

display(
    Markdown(
        ('<p style="font-size: 16px">当前剩余贷款本金是{}，{}个月等额本金法支付本息之和是{}。</p>'
        +'<p style="font-size: 16px">当前提前还款本金{}，还款手续费/利息{}。</p>'
        +'<p style="font-size: 16px">如果提前还款支付本息之和是{}。共计减少支付{}。</p>'
        +'<p style="font-size: 16px">如果将{}用于年化 {}% 的无风险投资。{}个月最终可获得的收益是{}（复利法）。</p> ').format(
            round(principal_before, 2),
            term,
            round(total_payments_before, 2),
            round(return_amount, 2),
            round(return_intrest, 2),
            round(total_payments_after, 2),
            round(total_payments_before - total_payments_after, 2),
            round(return_amount + return_intrest, 2),
            round(invest_intrest_year * 100, 2),
            term,
            round(
                (return_amount + return_intrest) * (1 + invest_intrest_month) ** term
                - (return_amount + return_intrest),
                2,
            ),
        )
    )
)

for i in range(term):
    interest_before = remain_before * rate_month
    remain_before = remain_before - decrease_before

    interest_after = remain_after * rate_month
    remain_after = remain_after - decrease_after

    payments_data.append(
        {
            "月": i + 1,
            "本金(前)": round(decrease_before, 2),
            "利息(前)": round(interest_before, 2),
            "月供(前)": round(decrease_before + interest_before, 2),
            "剩余本金(前)": round(remain_before, 2),
            "本金(后)": round(decrease_after, 2),
            "利息(后)": round(interest_after, 2),
            "月供(后)": round(decrease_after + interest_after, 2),
            "剩余本金(后)": round(remain_after, 2),
        }
    )

df = pd.DataFrame(payments_data)
display(
    df.style.format(
        {
            "本金(前)": "¥{:,.2f}",
            "利息(前)": "¥{:,.2f}",
            "月供(前)": "¥{:,.2f}",
            "剩余本金(前)": "¥{:,.2f}",
            "本金(后)": "¥{:,.2f}",
            "利息(后)": "¥{:,.2f}",
            "月供(后)": "¥{:,.2f}",
            "剩余本金(后)": "¥{:,.2f}",
        }
    )
)

<p style="font-size: 16px">当前剩余贷款本金是10000，36个月等额本金法支付本息之和是10601.25。</p><p style="font-size: 16px">当前提前还款本金5000，还款手续费/利息5。</p><p style="font-size: 16px">如果提前还款支付本息之和是10305.62。共计减少支付295.62。</p><p style="font-size: 16px">如果将5005用于年化 2.0% 的无风险投资。36个月最终可获得的收益是309.23（复利法）。</p> 

,月,本金(前),利息(前),月供(前),剩余本金(前),本金(后),利息(后),月供(后),剩余本金(后)
0,1,¥277.78,¥32.50,¥310.28,"¥9,722.22",¥138.89,¥16.25,¥155.14,"¥4,861.11"
1,2,¥277.78,¥31.60,¥309.38,"¥9,444.44",¥138.89,¥15.80,¥154.69,"¥4,722.22"
2,3,¥277.78,¥30.69,¥308.47,"¥9,166.67",¥138.89,¥15.35,¥154.24,"¥4,583.33"
3,4,¥277.78,¥29.79,¥307.57,"¥8,888.89",¥138.89,¥14.90,¥153.78,"¥4,444.44"
4,5,¥277.78,¥28.89,¥306.67,"¥8,611.11",¥138.89,¥14.44,¥153.33,"¥4,305.56"
5,6,¥277.78,¥27.99,¥305.76,"¥8,333.33",¥138.89,¥13.99,¥152.88,"¥4,166.67"
6,7,¥277.78,¥27.08,¥304.86,"¥8,055.56",¥138.89,¥13.54,¥152.43,"¥4,027.78"
7,8,¥277.78,¥26.18,¥303.96,"¥7,777.78",¥138.89,¥13.09,¥151.98,"¥3,888.89"
8,9,¥277.78,¥25.28,¥303.06,"¥7,500.00",¥138.89,¥12.64,¥151.53,"¥3,750.00"
9,10,¥277.78,¥24.38,¥302.15,"¥7,222.22",¥138.89,¥12.19,¥151.08,"¥3,611.11"


## 等额本息

In [90]:
monthly_before = (
    principal_before
    * rate_month
    * (1 + rate_month) ** term
    / ((1 + rate_month) ** term - 1)
)

monthly_after = (
    (principal_before - return_amount)
    * rate_month
    * (1 + rate_month) ** term
    / ((1 + rate_month) ** term - 1)
)

total_payments_before = monthly_before * term
total_payments_after = monthly_after * term + return_amount + return_intrest

display(
    Markdown(
        ('<p style="font-size: 16px">当前剩余贷款本金是{}，{}个月等额本息法支付本息之和是{}。</p>'
        +'<p style="font-size: 16px">当前提前还款本金{}，还款手续费/利息{}。</p>'
        +'<p style="font-size: 16px">如果提前还款支付本息之和是{}。共计减少支付{}。</p>'
        +'<p style="font-size: 16px">如果将{}用于年化 {}% 的无风险投资。{}个月最终可获得的收益是{}（复利法）。</p> ').format(
            round(principal_before, 2),
            term,
            round(total_payments_before, 2),
            round(return_amount, 2),
            round(return_intrest, 2),
            round(total_payments_after, 2),
            round(total_payments_before - total_payments_after, 2),
            round(return_amount + return_intrest, 2),
            round(invest_intrest_year * 100, 2),
            term,
            round(
                (return_amount + return_intrest) * (1 + invest_intrest_month) ** term
                - (return_amount + return_intrest),
                2,
            ),
        )
    )
)

remaining_before = principal_before
remaining_after = principal_before - return_amount
payments_data = []
for month in range(term):
    interest_before = remaining_before * rate_month
    decrease_before = monthly_before - interest_before
    remaining_before -= decrease_before

    interest_after = remaining_after * rate_month
    decrease_after = monthly_after - interest_after
    remaining_after -= decrease_after

    payments_data.append(
        {
            "月": month + 1,
            "利息(前)": interest_before,
            "本金(前)": decrease_before,
            "月供(前)": interest_before + decrease_before,
            "剩余本金(前)": remaining_before,
            "利息(后)": interest_after,
            "本金(后)": decrease_after,
            "月供(后)": interest_after + decrease_after,
            "剩余本金(后)": remaining_after,
        }
    )

df = pd.DataFrame(payments_data)
display(
    df.style.format(
        {
            "本金(前)": "¥{:,.2f}",
            "利息(前)": "¥{:,.2f}",
            "月供(前)": "¥{:,.2f}",
            "剩余本金(前)": "¥{:,.2f}",
            "本金(后)": "¥{:,.2f}",
            "利息(后)": "¥{:,.2f}",
            "月供(后)": "¥{:,.2f}",
            "剩余本金(后)": "¥{:,.2f}",
        }
    )
)

<p style="font-size: 16px">当前剩余贷款本金是10000，36个月等额本息法支付本息之和是10612.63。</p><p style="font-size: 16px">当前提前还款本金5000，还款手续费/利息5。</p><p style="font-size: 16px">如果提前还款支付本息之和是10311.31。共计减少支付301.31。</p><p style="font-size: 16px">如果将5005用于年化 2.0% 的无风险投资。36个月最终可获得的收益是309.23（复利法）。</p> 

,月,利息(前),本金(前),月供(前),剩余本金(前),利息(后),本金(后),月供(后),剩余本金(后)
0,1,¥32.50,¥262.30,¥294.80,"¥9,737.70",¥16.25,¥131.15,¥147.40,"¥4,868.85"
1,2,¥31.65,¥263.15,¥294.80,"¥9,474.56",¥15.82,¥131.57,¥147.40,"¥4,737.28"
2,3,¥30.79,¥264.00,¥294.80,"¥9,210.55",¥15.40,¥132.00,¥147.40,"¥4,605.28"
3,4,¥29.93,¥264.86,¥294.80,"¥8,945.69",¥14.97,¥132.43,¥147.40,"¥4,472.85"
4,5,¥29.07,¥265.72,¥294.80,"¥8,679.97",¥14.54,¥132.86,¥147.40,"¥4,339.99"
5,6,¥28.21,¥266.59,¥294.80,"¥8,413.39",¥14.10,¥133.29,¥147.40,"¥4,206.69"
6,7,¥27.34,¥267.45,¥294.80,"¥8,145.93",¥13.67,¥133.73,¥147.40,"¥4,072.97"
7,8,¥26.47,¥268.32,¥294.80,"¥7,877.61",¥13.24,¥134.16,¥147.40,"¥3,938.81"
8,9,¥25.60,¥269.19,¥294.80,"¥7,608.42",¥12.80,¥134.60,¥147.40,"¥3,804.21"
9,10,¥24.73,¥270.07,¥294.80,"¥7,338.35",¥12.36,¥135.03,¥147.40,"¥3,669.18"
